In [ ]:
In this weeks assignment you will gain practical experience in working with both data and statistical models, the most important skillset you should retain from this course. You can solve the assignment in your programming language of choice, but should hand in the solution as a runnable file (e.g. an R-script, a Python-script, a Quarto-file, a Jupyter notebook etc). Ensure that your file can be run from top to bottom without any modifications, which is not necessarily the case if you tend to run the script line-by-line and jump back and forth while programming. If you are unable to solve any of the tasks contact me by email and I can help out. If you are unable to do so before the deadline, reflect and write a brief summary on why you think you are seeing the problems you are seeing.
Exercise 1: Preparations

    Download the Auto.csv dataset from the ISLP website (https://www.statlearning.com/resources-python 

    Lenker til en ekstern side. or https://www.statlearning.com/resources-second-edition). This you can do outside Python or R, but from here on out the remainder of the solution should be programmed.
    Read the Auto.csv-dataset into memory.
    In the horsepower-column, some values are missing. These are encoded with ‘?’. Remove these rows from the dataset.
    Create a new column ‘muscle’. This column should contain a 1 for all muscle cars (e.g. cars that have above average horsepower) and 0 for the rest. (HINT: To help you out I have calculated that you should end up with approximately 148 muscle cars and 244 others (assuming that you have already removed the rows with horsepower=’?’ above)).
    Split the dataset into a training set and a test set, by randomly drawing 80% of the rows for the former and 20% of the rows for the latter. We will get into why this is a good idea in later lectures, but its good practice to start with evaluating the models in held-out data from the get go.

Exercise 2: Simple linear regression

    Fit a simple linear regression model using horsepower as the predictor and mpg as the outcome using the training data. 
    Create a scatter plot with horsepower on the x-axis and mpg on the y-axis using the testing data. Plot the regression line found by the model in the plot (Hint: To achieve this you need to find the intercept and the single coefficient of the model, generate predictions or use built-in plotting functionality).
    Use the model to generate predictions for the training set. Calculate and report the mean absolute error (MAE) of these predictions.
    Use the model to generate predictions for the test set. Calculate and report the MAE of the predictions.
    Reflection: Is the training or testing MAE is lower? Does this match your expectation? What would be the general pattern we expect here (e.g. one is lower than the other, they are the same, ...), and why do we expect that?

Exercise 3: Multivariate linear regression

    Fit a multivariate linear regression model using horsepower, weight, displacement, and year as predictors and mpg as the outcome.
    Print the intercept and coefficients of the model (these should be identifiable such that I’m able to deduce which coefficient belongs to which variable). What can you say about the relationship between mpg and the year-variable based on these?
    Use the model to generate predictions for the training set. Calculate and report the MAE of these predictions.
    Use the model to generate predictions for the test set. Calculate and report the MAE of the predictions.
    Reflection: Is the training MAE lower or higher than in the simple linear regression model? Does it have to be this way, or could it have been otherwise? What about the testing MAE?

Exercise 4: Logistic regression

    Fit a logistic regression model using weight, displacement and year as predictors and our newly created muscle-column as the outcome. Why don’t we use horsepower as a predictor in this model?
    Use the model to generate predictions for the training set. Calculate and report the accuracy of these predictions.
    Use the model to generate predictions for the testing set. Calculate and report the accuracy of these predictions.
